<a href="https://colab.research.google.com/github/KevinBarranco24/Practica_4_Dash/blob/main/Practica_4_Dashoboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Práctica 4
##Dashboard

###Kevin Kristop Morales Barranco - 214603492



#MODULES

In [1]:
import numpy as np
import pandas as pd
import math
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

#CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#FUNCTIONS

##Brownian Motion

In [3]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  #n_steps = 1000
  #s_pos = [0,0]
  #speed = 6

  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    #turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df


In [4]:
BM_2d_df_2 = bm_2d(2000, 9, [5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
                      x = BM_2d_df_2.x_pos,
                      y = BM_2d_df_2.y_pos,
                      marker = dict(size=2),
                      line = dict(width=1),
                      mode='lines',
                      name='BM_2d_trajectory',
                      showlegend=True
))

fig_BM_2d.show()

##Correlated Random Walks

In [5]:
def CRW(CRW_exponents=0.2, resolution=800, speed=6, s_pos=[0,0]):
  #init variables

  #resolution = 1000
  #s_pos = [0,0]
  #speed = 6

  #init arrays
  aux_domain = np.linspace(0, 2*np.pi, resolution)

  #init velocity vector
  velocity = Vec2d(speed, 0)

  wrapcauchy_rvs = wrapcauchy.rvs(CRW_exponents, size=resolution)


  #init DataFrames
  CRW_2d_df_s = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)

  for j in range(resolution):
    velocity = velocity.rotated(wrapcauchy_rvs[j])

    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df_s.x_pos[j]+velocity.x,'y_pos':CRW_2d_df_s.y_pos[j]+velocity.y}])
    CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)


  return CRW_2d_df_s
  #print(CRW_2d_df)


In [6]:
#print(CRW(0.5,16,3,[0,0]))
BM_2d_df = CRW(0.8,500,5,[5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_2d_df.x_pos,
    y = BM_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='CRW_trajectory',
    showlegend=True
))

fig_BM_2d.show()

##Levy Walks

In [7]:
def levy(Levy_exponent=0.7, miu=3.0, beta=0, n_steps=250, s_pos=[0,0]):
  #Init variables
  #Levy_exponent = 0.7
  #miu = 3.0
  #beta = 0
  #n_steps = 250
  #s_pos = [0,0]

  Levy_rvs = levy_stable.rvs(Levy_exponent,beta, loc=miu, size=n_steps)

  Wrapcauchy_rvs = wrapcauchy.rvs(c=Levy_exponent, loc=beta, size=n_steps)
  cauchy_rvs = Wrapcauchy_rvs
  for n in range(len(cauchy_rvs)):
    for m in range(n):
      cauchy_rvs[n] += cauchy_rvs[m]

  Levy_rvs_abs = abs(Levy_rvs)

  LF_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])

  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])

  LF_2d_df = pd.concat([LF_2d_df,temp_df], ignore_index=True)

  for i in range(n_steps-1):
    #init velocity vector
    velocity = Vec2d(Levy_rvs_abs[i], 0)
    velocity = velocity.rotated(cauchy_rvs[i])

    temp_df = pd.DataFrame([{'x_pos':LF_2d_df.x_pos[i]+velocity.x,'y_pos':LF_2d_df.y_pos[i]+velocity.y}])
    LF_2d_df = pd.concat([LF_2d_df,temp_df], ignore_index=True)
  return LF_2d_df

In [8]:
LF_2d_df = levy(0.91, 3.0, 0, 1000, [5,7])
fig_LF_2d = go.Figure()

fig_LF_2d.add_trace(go.Scatter(x= LF_2d_df.x_pos,
                              y= LF_2d_df.y_pos,
                              #z=np.linspace(0,1,n_steps),
                              marker= dict(size=2),
                              line=dict(width=1),
                              mode='lines',
                              name='LF_3d',
                              showlegend=True))

#fig_LF_2d.update_layout(title_text='Levy flight in 3D',
                        #autosize=False,
                       # width = 900,
                       # height = 700)

fig_LF_2d.show()

##Path Length

In [9]:
def path_length(df = pd.DataFrame):
  n_df = np.array([distance.euclidean(df.loc[i-1], df.iloc[i]) for i in range(1,df.shape[0])])
  return np.cumsum(n_df)

In [10]:
BM_2d_3 = bm_2d(1000, 3, [5,6])
BM_2d_6 = bm_2d(1000, 6, [7,9])
CRW_2d_6 = CRW(0.94,1000,6,[3,7])

BM_3 = path_length(BM_2d_3)
BM_6 = path_length(BM_2d_6)
CRW_6 = path_length(CRW_2d_6)


In [11]:
#init plot
fig_path_length = go.Figure()

#add results to the plot
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(BM_3))+1,
    y = BM_3,
    marker = dict(size=25),
    line = dict(width=25),
    mode = 'lines',
    name = 'path_length_BM_3',
    showlegend = True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(BM_6))+1,
    y = BM_3,
    marker = dict(size=13),
    line = dict(width=12),
    mode = 'lines',
    name = 'path_length_BM_6',
    showlegend = True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(CRW_6))+1,
    y = BM_3,
    marker = dict(size=4),
    line = dict(width=4),
    mode = 'lines',
    name = 'path_length_CRW_6',
    showlegend = True
))

fig_path_length.show()

##Mean Squared Displacement

In [12]:
def msd(df = pd.DataFrame):
  msd_2 = np.empty(shape=(0))
  list_msd = []

  for i in range(1,df.shape[0]):
    disp_vec = np.array([distance.euclidean(df.iloc[n-i], df.iloc[n]) for n in range(i, df.shape[0],1)])
    list_msd.append(np.sum(np.power(disp_vec, 2))/len(disp_vec))

  msd_2 = np.asarray(list_msd)

  return msd_2

In [13]:
#init walks
BM_2d_6 = bm_2d(100, 6, [0,0])
CRW_2d_6 = CRW(0.4,100,6,[0,0])

In [14]:
BM_6 = msd(BM_2d_6)
CRW_6 = msd(CRW_2d_6)

#Saving metricts in Dataframe
BM_df = pd.DataFrame(BM_6)
CRW_df = pd.DataFrame(CRW_6)

In [15]:
#init figure

fig_msd = go.Figure()

fig_msd.add_trace(go.Scatter(
    x = np.arange(len(BM_6)),
    y = BM_6,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_BM_6',
    showlegend = True
))

fig_msd.add_trace(go.Scatter(
    x = np.arange(len(CRW_6)),
    y = CRW_6,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_CRW_6',
    showlegend = True
))

fig_msd.show()

#Dashboard

In [36]:
opcion = pnw.RadioButtonGroup(name='opcion', options=['BM','CRW','LF'])
n_steps = pnw.IntSlider(name='Number of steps', width=200, value=0, step=20, start=0, end=1000)
x_pos = pnw.IntInput(name='X position', value=0, step=1, start=1, end=25)
y_pos = pnw.IntInput(name='Y position', value=0, step=1, start=1, end=25)
speed = pnw.IntSlider(name='Speed', width=200, value=0, step=1, start=0, end=100)
metric = pnw.Select(name='Metric', value='Path Length', options=['Path Length','MSD'])


pn.Column(opcion, n_steps, x_pos, y_pos, speed, metric, height=200)


Column(height=200)
    [0] RadioButtonGroup(name='opcion', options=['BM', 'CRW', 'LF'], value='BM')
    [1] IntSlider(end=1000, name='Number of steps', step=20, width=200)
    [2] IntInput(end=25, name='X position', start=1)
    [3] IntInput(end=25, name='Y position', start=1)
    [4] IntSlider(end=100, name='Speed', width=200)
    [5] Select(name='Metric', options=['Path Length', 'MSD'], value='Path Length')